In [1]:
import os
import sys
import time
import math
import shutil

import numpy as np
from tqdm import tqdm

pyngp_path = '/cluster/home/jpostels/nnaimi/instant-ngp/build'
sys.path.append(pyngp_path)

sys.path.append(os.path.abspath("/cluster/home/jpostels/nnaimi/instant-ngp/scripts"))

import pyngp as ngp

from common import *
from scenes import *
from quantize import *

from utils import deserialize_ingp

In [2]:
snapshot = '/cluster/work/cvl/jpostels/nnaimi/data/kodim11-20230721-232433.ingp'
# snapshot = '/cluster/work/cvl/jpostels/nnaimi/data/snapshot_8bit_params.ingp'

In [3]:
!nvidia-smi
!pwd
!echo $CUDA_VISIBLE_DEVICES
# !printenv | grep python

Mon Sep  4 10:34:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:21:00.0 Off |                  N/A |
|  0%   26C    P8    27W / 370W |      1MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# !ls -alh /cluster/work/cvl/jpostels/nnaimi/data

In [5]:
snapshot_dir = '/cluster/work/cvl/jpostels/nnaimi/output'
quantized_save_dir = '/cluster/work/cvl/jpostels/nnaimi/snapshots/kodim11_base_res_quantized'

scene = '/cluster/work/cvl/jpostels/nnaimi/data/KODAK/kodim11.png'
# network_config = '/cluster/home/jpostels/nnaimi/instant-ngp/configs/image/benchmarking.json'

In [7]:
import glob
from datetime import datetime

start_time_str = '20230729-204000'
end_time_str = '20230730-005800'

start_time = datetime.strptime(start_time_str, '%Y%m%d-%H%M%S')
end_time = datetime.strptime(end_time_str, '%Y%m%d-%H%M%S')

all_files = glob.glob(os.path.join(snapshot_dir, 'kodim11-*'))

matching_files = []
for file_path in all_files:
    # Extract the datetime from the filename
    file_name = os.path.basename(file_path)[len('kodim11-'):]
    file_time_str = file_name.split('.')[0]  # Assuming the filename doesn't contain dots
    file_time = datetime.strptime(file_time_str, '%Y%m%d-%H%M%S')

    # Check if the file's datetime falls within the specified range
    if start_time <= file_time <= end_time:
        matching_files.append(file_path)

# Print the list of matching files
# for file in matching_files:
#     print(file)

print(len(matching_files))

0


In [ ]:
# for snapshot_path in matching_files:
    # quantize_snapshot(snapshot_path, save_dir=quantized_save_dir, bits=8, zero_point_rounding=True)

In [6]:
# render stuff
argswidth = 768
argsheight = 512

# training
argstrain = False
argsn_steps = 0

argsnerf_compatibility = False

argssave_snapshot = ""
argstest_transforms = ""

# camera
argssharpen = 0.0
argsexposure = 0.0

# screenshot args
argsscreenshot_transforms = ""
argsscreenshot_frames = None
argsscreenshot_dir = "/cluster/work/cvl/jpostels/nnaimi/renders"
argsscreenshot_spp = 1 # default is 16

# mse = float(compute_error("MSE", A, R))
# ssim = float(compute_error("SSIM", A, R))

def get_scene(scene):
    for scenes in [scenes_sdf, scenes_nerf, scenes_image, scenes_volume]:
        if scene in scenes:
            return scenes[scene]
    return None

In [7]:
# snapshot = '/cluster/work/cvl/jpostels/nnaimi/data/kodim11-20230721-232433.ingp'
# snapshot = '/cluster/work/cvl/jpostels/nnaimi/data/snapshot_7bit_params.ingp'


snapshot_dir = '/cluster/work/cvl/jpostels/nnaimi/snapshots/linear_quantization'
# snapshot = snapshot_dir + '/snapshot_nozpx_round_7bit_params.ingp'
# snapshot = snapshot_dir + '/snapshot_8bit_params.ingp'
# snapshot = '/cluster/work/cvl/jpostels/nnaimi/snapshots/pca_study_quantized' + "/kodim11-20230802-225729_8bit.ingp"
# snapshot = '/cluster/work/cvl/jpostels/nnaimi/snapshots/pca_study_quantized' + "/kodim11-20230802-225729_16bit.ingp"
snapshot = '/cluster/work/cvl/jpostels/nnaimi/output' + "/kodim11-20230802-225729.ingp"

In [36]:
base_dir = '/cluster/work/cvl/jpostels/nnaimi'

quantized_dir = base_dir + '/snapshots/pca_study_quantized'
og_dir = base_dir + '/output'

In [32]:
import pandas as pd

quantized_snapshots = glob.glob(os.path.join(quantized_dir, '*'))

header = ['snapshot_name', 'n_enc_params', 'n_params', 'log2_hashmap_size', 'n_levels', 'per_level_scale', 'base_resolution', 'n_features_per_level', 'mse', 'clipped_mse', 'psnr', 'ssim', 'og_psnr', 'bpp']
num_rows = len(quantized_snapshots)
results_quant = pd.DataFrame(columns=header, index=range(num_rows))

In [33]:
results_quant.iloc[0]['snapshot_name']

nan

In [ ]:
for ind, snapshot in enumerate(quantized_snapshots):
    # print(">>"*10, snapshot)
    testbed = ngp.Testbed()
    testbed.root_dir = ROOT_DIR

    if scene:
        scene_info = get_scene(scene)
        if scene_info is not None:
            scene = os.path.join(scene_info["data_dir"], scene_info["dataset"])
            if not network_config and "network" in scene_info:
                network_config = scene_info["network"]
        testbed.load_training_data(scene)

    if snapshot:
        scene_info = get_scene(snapshot)
        if scene_info is not None:
            snapshot = default_snapshot_filename(scene_info)
        testbed.load_snapshot(snapshot)
    # elif network_config:
    #     testbed.reload_network_from_file(network_config)

    testbed.nerf.sharpen = float(argssharpen)
    testbed.exposure = argsexposure
    testbed.shall_train = argstrain
    testbed.nerf.render_with_lens_distortion = True

    # network_stem = os.path.splitext(os.path.basename(network_config))[0] if network_config else "base"
    network_stem = 'quant'

    old_training_step = 0
    n_steps = argsn_steps

    # If we loaded a snapshot, didn't specify a number of steps, _and_ didn't open a GUI,
    # don't train by default and instead assume that the goal is to render screenshots,
    # compute PSNR, or render a video.
    if n_steps < 0 and (not snapshot or argsgui):
        n_steps = 35000

    tqdm_last_update = 0
    if n_steps > 0:
        print("n_steps")
        with tqdm(desc="Training", total=n_steps, unit="steps") as t:
            while testbed.frame():
                if testbed.want_repl():
                    repl(testbed)
                # What will happen when training is done?
                if testbed.training_step >= n_steps:
                    if argsgui:
                        testbed.shall_train = False
                    else:
                        break

                # Update progress bar
                if testbed.training_step < old_training_step or old_training_step == 0:
                    old_training_step = 0
                    t.reset()

                now = time.monotonic()
                if now - tqdm_last_update > 0.1:
                    t.update(testbed.training_step - old_training_step)
                    t.set_postfix(loss=testbed.loss)
                    old_training_step = testbed.training_step
                    tqdm_last_update = now

    if argssave_snapshot:
        os.makedirs(os.path.dirname(argssave_snapshot), exist_ok=True)
        testbed.save_snapshot(argssave_snapshot, False)

    if argsscreenshot_dir:
        snapshot_name = os.path.basename(snapshot)[:-len(".ingp")]
        outname = os.path.join(argsscreenshot_dir, snapshot_name + "_" + network_stem)
        print(f"Rendering {outname}.png")

        testbed.render_ground_truth = True
        ref_image = testbed.render(argswidth or 1920, argsheight or 1080, 1, True)

        testbed.render_ground_truth = False    
        image = testbed.render(argswidth or 1920, argsheight or 1080, argsscreenshot_spp, True)

        A = np.clip(linear_to_srgb(image[...,:3]), 0.0, 1.0)
        R = np.clip(linear_to_srgb(ref_image[...,:3]), 0.0, 1.0)
        mse = float(compute_error("MSE", A, R))
        clipped_mse = float(compute_error("MScE", A, R))
        psnr = mse2psnr(mse)
        ssim = float(compute_error("SSIM", A, R))

        print(clipped_mse, mse, psnr, ssim)
        

        if os.path.dirname(outname) != "":
            os.makedirs(os.path.dirname(outname), exist_ok=True)

        write_image(outname + ".png", image)
        write_image(outname + "_reference.png", ref_image)
        
        
        # All logging
        og_filename = os.path.basename(snapshot)[:-len("_8bit.ingp")] + ".ingp"       
        config = deserialize_ingp(og_dir + "/" + og_filename)
        
        results_quant.iloc[ind]['snapshot_name'] = os.path.basename(snapshot)       
        results_quant.iloc[ind]['n_enc_params'] = testbed.n_encoding_params()
        results_quant.iloc[ind]['n_params'] = testbed.n_params()
        results_quant.iloc[ind]['log2_hashmap_size'] = config['encoding']['log2_hashmap_size']
        results_quant.iloc[ind]['n_levels'] = config['encoding']['n_levels']
        results_quant.iloc[ind]['per_level_scale'] = config['encoding']['per_level_scale']
        results_quant.iloc[ind]['base_resolution'] = config['encoding']['base_resolution']
        results_quant.iloc[ind]['n_features_per_level'] = config['encoding']['n_features_per_level']
        results_quant.iloc[ind]['mse'] = mse
        results_quant.iloc[ind]['clipped_mse'] = clipped_mse
        results_quant.iloc[ind]['psnr'] = psnr
        results_quant.iloc[ind]['ssim'] = ssim
        results_quant.iloc[ind]['og_psnr'] = -10.0 * np.log(config['snapshot']['loss']) / np.log(10.0) # mse2psnr(config['snapshot']['loss'])
        results_quant.iloc[ind]['bpp'] = float(testbed.n_params() * 8) / (argswidth * argsheight)

15:30:21 SUCCESS  Initialized CUDA 11.4. Active GPU is #0: NVIDIA GeForce RTX 3090 [86]
15:30:21 INFO     Loading STBI image from /cluster/work/cvl/jpostels/nnaimi/data/KODAK/kodim11.png
15:30:21 SUCCESS  Loaded a full-precision image with 768x512 pixels.
Rendering /cluster/work/cvl/jpostels/nnaimi/renders/kodim11-20230802-233655_8bit_quant.png
15:30:21 INFO     Loading network snapshot from: /cluster/work/cvl/jpostels/nnaimi/snapshots/pca_study_quantized/kodim11-20230802-233655_8bit.ingp
15:30:21 INFO     GridEncoding:  Nmin=173 b=1.6 F=8 T=2^4 L=11
15:30:21 INFO     Model:         2--[HashGrid]-->96--[FullyFusedMLP(neurons=64,layers=4)]-->3
15:30:21 INFO       total_encoding_params=1408 total_network_params=11264
0.15354035794734955 0.15354035794734955 8.13777451260905 0.002373156137764454
15:30:22 SUCCESS  Initialized CUDA 11.4. Active GPU is #0: NVIDIA GeForce RTX 3090 [86]
15:30:22 INFO     Loading STBI image from /cluster/work/cvl/jpostels/nnaimi/data/KODAK/kodim11.png
15:30:22 S

/cluster/home/jpostels/nnaimi/instant-ngp/scripts/common.py:145: RuntimeWarning: invalid value encountered in power
  return np.where(img > limit, 1.055 * (img ** (1.0 / 2.4)) - 0.055, 12.92 * img)


0.0006992113776504993 0.0006992113776504993 31.553915134248165 0.9040783047676086
15:30:23 SUCCESS  Initialized CUDA 11.4. Active GPU is #0: NVIDIA GeForce RTX 3090 [86]
15:30:23 INFO     Loading STBI image from /cluster/work/cvl/jpostels/nnaimi/data/KODAK/kodim11.png
15:30:23 SUCCESS  Loaded a full-precision image with 768x512 pixels.
15:30:23 INFO     Loading network snapshot from: /cluster/work/cvl/jpostels/nnaimi/snapshots/pca_study_quantized/kodim11-20230802-181854_8bit.ingp
Rendering /cluster/work/cvl/jpostels/nnaimi/renders/kodim11-20230802-181854_8bit_quant.png
15:30:23 INFO     GridEncoding:  Nmin=174 b=1.4 F=1 T=2^8 L=2
15:30:23 INFO     Model:         2--[HashGrid]-->16--[FullyFusedMLP(neurons=64,layers=4)]-->3
15:30:23 INFO       total_encoding_params=512 total_network_params=6144
0.027980215847492218 0.027980215847492218 15.531489395577998 0.3674793541431427
15:30:24 SUCCESS  Initialized CUDA 11.4. Active GPU is #0: NVIDIA GeForce RTX 3090 [86]
15:30:24 INFO     Loading ST

In [37]:
results_quant.to_csv(base_dir + "/results_quantization_pca_study.csv", index=False)